### Required Libraries:

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotSelectableException
from selenium.common.exceptions import ElementNotInteractableException

import time
#import venue
import pandas as pd
s=Service(ChromeDriverManager().install())

In [2]:
import bs4 as bs
import datetime
from datetime import datetime
from datetime import date
import email_sending as es
import spac_to_db_funs as dbf

### Introdcution of the page for extraction:

In [3]:
driver =webdriver.Chrome(service=s)
driver.get('https://nftpricefloor.com/')
extracted_text = ''
extracted_inner_html = ''
extracted_number_of_pages = 0  

### Function to perform the extraction 

In [4]:
#__next > div.container > main > div > table
#__next > div.container > main > div > table > tbody
def perform_extraction():
    delay = 10 # seconds
    try:       
        target =  WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR , "#__next > div.container > main > div > table")))

    except TimeoutException:
        print ("Loading took too much time!, number of extracted pages: 0")
        
    
    return (target.text, target.get_attribute('innerHTML'))

### Move the scroll bar so the new items get loaded

In [5]:
for i in range(0, 10):
    extracted_text = extracted_text + perform_extraction()[0]
    extracted_inner_html = extracted_inner_html + perform_extraction()[1]
    driver.execute_script("window.scrollBy(0, 500)")
    time.sleep(1)
    i=i+1
extracted_number_of_pages = extracted_number_of_pages +1

### Function to organize the extracted info into rows, headers, image links and 

In [6]:
def rows_extractor(extracted_inner_html ):
    data = []
    soup = bs.BeautifulSoup(extracted_inner_html, "html.parser")
    rows=soup.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        #data.append([ele for ele in cols if ele]) # Get rid of empty values
        
        temp_list= [ele for ele in cols if ele] # Get rid of empty values
        if temp_list not in data:
            data.append(temp_list)
            

    return  data[1:]

In [7]:
def img_extractor2(extracted_inner_html):
    soup = bs.BeautifulSoup(extracted_inner_html, "html.parser")
    rows_test=soup.find_all('tr')
    images= []
    for r in rows_test:
        
        try:
            
            tmp_name = r.find('a').find_all('img')[1]['src']
            if 'data:image/' not in tmp_name:
                tmp_image = 'https://nftpricefloor.com'+ tmp_name
                if tmp_image not in images:
                    images.append(tmp_image)
        except:
            pass
    
    return  images

In [8]:
def headers_extractor(extracted_inner_html):
    soup = bs.BeautifulSoup(extracted_inner_html, "html.parser")
    rows=soup.find_all('tr')
    
    COLS = rows[0].find_all('th')
    COLS = [ele.text.strip() for ele in COLS]
    data_column =[]
    data_column.append([ele for ele in COLS if ele])
    
    return data_column[0] 

In [9]:
delay = 10 # seconds
button = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CSS_SELECTOR , "a[aria-label='Next page']")))
target = WebDriverWait(driver,delay).until(EC.visibility_of_element_located ((By.CSS_SELECTOR , "#__next > div.container > main > div > table")))

In [10]:
if( button.is_enabled() and target.is_displayed()):
    button.click()
    time.sleep(2)
else:
    print('Button is not clickable anymore')

### Move through all the pages:

In [11]:
extraction_going_on = 1
while extraction_going_on != 0:
    
    try:       

        
        for i in range(0, 11):
            extracted_text = extracted_text + perform_extraction()[0]
            extracted_inner_html = extracted_inner_html + perform_extraction()[1]
            driver.execute_script("window.scrollBy(0, 500)")
            time.sleep(1)
            i=i+1
        extracted_number_of_pages = extracted_number_of_pages +1
    except TimeoutException:
        print ("Loading took too much time!, number of extracted pages: ???")
        extraction_going_on = 0

    if( extracted_number_of_pages!=10):
        button.click()
        time.sleep(2)
    else:
        #print('Button is not clickable anymore and extraction stoped')
        #print('Number of extracted pages:')
        #print(extracted_number_of_pages)
        extraction_going_on = 0
        #break

In [12]:
rows = rows_extractor(extracted_inner_html ) 

In [13]:
COLS = headers_extractor(extracted_inner_html)

In [14]:
images = img_extractor2(extracted_inner_html)

### Storing the extracted data into list objects

In [15]:
Id = [ int(elem[0])  for elem in rows]
collectible = [ elem[1] for elem in rows]
price_floor =  [ elem[2] for elem in rows]
t4_pct = [ elem[3] for elem in rows ]
vol_24 = [ elem[4] for elem in rows ]
sales_24 = [ int(elem[5]) for elem in rows ]

In [16]:
makrket_cap = [ elem[8] for elem in rows ] 

### Establish a Panda df from the extracted lists:

In [17]:
df_collections = pd.DataFrame()

In [18]:
try:
    df_collections = pd.DataFrame( 
        {'created_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
         "visible_id": Id ,
         'collection_name' : collectible,
         'floor_price' : price_floor,
         'volume' : vol_24,
          'sales' : sales_24,
         'mark_cap': makrket_cap,
         'image_link' : images
        })
except:
    subject = 'URGENT, NFT Extraction failed'
    body = "NFT Extraction failed, and datafarme couldn't be established, pls check the process"
    #es.send_mail('------@yahoo.com', '-------i@gmail.com', subject, body)
    
    

### df datatype allocation, and text cleaning for better look:

In [19]:
def string_to_number (txt, decimal=2):
    txt = txt.replace(",", "")
    if (decimal == 0):
        return int (txt.split('ETH') [0]) 
    else:
        return round( float (txt.split('ETH') [0]) , decimal )

In [20]:
def string_to_int (txt):
    txt = txt.replace(",", "")    
    return int (txt.split('ETH') [0]) 

In [21]:
try:
    df_collections.drop_duplicates( subset="collection_name", keep='first', inplace= True,ignore_index = True)
    df_collections['floor_price'] = df_collections['floor_price'].map(string_to_number)
    df_collections['volume'] = df_collections['volume'].map(string_to_number)
    df_collections['mark_cap'] = df_collections['mark_cap'].map(string_to_int)
    df_collections['collection_name'] = df_collections['collection_name'].apply(dbf.chr_replacer)
except:
    subject = 'URGENT, NFT Dataframe editing failed'
    body = "NFT Dataframe couldn't be edited,  pls check the process"
    es.send_mail('------g@yahoo.com', '-------i@gmail.com', subject, body)
    
    

In [23]:
df_collections

,created_date,visible_id,collection_name,floor_price,volume,sales,mark_cap,image_link
0,2022-11-11 12:35:00,1,CryptoPunks,66.00,499.52,7,660000,https://nftpricefloor.com/_next/image?url=http...
1,2022-11-11 12:35:00,2,Bored Ape Yacht Club,58.46,2197.60,33,584600,https://nftpricefloor.com/_next/image?url=http...
2,2022-11-11 12:35:00,3,The Sandbox,1.27,25.37,18,211216,https://nftpricefloor.com/_next/image?url=http...
3,2022-11-11 12:35:00,4,Mutant Ape Yacht Club,10.60,678.64,62,205905,https://nftpricefloor.com/_next/image?url=http...
4,2022-11-11 12:35:00,5,Autoglyphs,325.00,0.00,0,166400,https://nftpricefloor.com/_next/image?url=http...
...,...,...,...,...,...,...,...,...
483,2022-11-11 12:35:00,484,Laura_Dumitriu,0.02,0.08,3,27,https://nftpricefloor.com/_next/image?url=http...
484,2022-11-11 12:35:00,485,tirli,0.01,0.04,2,27,https://nftpricefloor.com/_next/image?url=http...
485,2022-11-11 12:35:00,486,eando9745,0.01,0.11,7,21,https://nftpricefloor.com/_next/image?url=http...
486,2022-11-11 12:35:00,487,oana193,0.01,0.38,2,20,https://nftpricefloor.com/_next/image?url=http...
